In [2]:
# load required packages
import pandas as pd
import sklearn as sk
import numpy as np
# from google.colab import files
import math

In [3]:
# install vowpal wabbit
!pip install boost
!apt-get install libboost-program-options-dev zlib1g-dev libboost-python-dev -y
!pip install vowpalwabbit

  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 1.6 MB 11.4 MB/s            
     |████████████████████████████████| 147 kB 49.9 MB/s            
  Created wheel for boost: filename=boost-0.1-py3-none-any.whl size=12462 sha256=f9663f36251ba0d16085e20fe50c82710e20fbc6be4f20b6e09904140738a427
  Stored in directory: /home/arshita/.cache/pip/wheels/f6/54/5c/a0275439e4f26dc996886e3a9ab5af36fd209fb048ea42ca76
Successfully built boost
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
     |████████████████████████████████| 3.0 MB 9.3 MB/s            


In [ ]:
files.upload()

In [4]:
df = pd.read_csv('Datasets/reward.csv')
df.head()

,Date,Name of State / UT,Total Confirmed cases (Indian National),Total Confirmed cases ( Foreign National ),Cured/Discharged/Migrated,Death,Total Confirmed cases,%share of infected cases,1 / (% share of infected cases),Population Share,(Population share) / (% share of infected cases),Recovery Rate,Death Rate,Population,Infection Rate,Susceptible Population,Rate of change of susceptible population,Action,Reward
0,10-Mar-20,Delhi,4,0,0,0,4,66.666667,0.015000,8.594458,0.128917,0.0,0.0,16787941,0.000024,16787937,0.000000,0,0.0
1,10-Mar-20,Maharashtra,2,0,0,0,2,33.333333,0.030000,57.529182,1.725875,0.0,0.0,112374333,0.000002,112374331,0.000000,0,0.0
2,11-Mar-20,Delhi,5,0,0,0,5,83.333333,0.012000,8.594458,0.103134,0.0,0.0,16787941,0.000030,16787936,-0.000006,1,0.0
3,11-Mar-20,Maharashtra,2,0,0,0,2,33.333333,0.030000,57.529182,1.725875,0.0,0.0,112374333,0.000002,112374331,0.000000,0,0.0
4,12-Mar-20,Delhi,6,0,0,0,6,35.294118,0.028333,8.594458,0.243510,0.0,0.0,16787941,0.000036,16787935,-0.000006,1,0.0


In [5]:
for i in range(len(df)):
  vaccineAmt = df['Susceptible Population'].iloc[i]*100/df['Population'].iloc[i]
  df['Action'].iloc[i] = vaccineAmt
df.tail()

/home/arshita/anaconda3/envs/py36_ai/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,Date,Name of State / UT,Total Confirmed cases (Indian National),Total Confirmed cases ( Foreign National ),Cured/Discharged/Migrated,Death,Total Confirmed cases,%share of infected cases,1 / (% share of infected cases),Population Share,(Population share) / (% share of infected cases),Recovery Rate,Death Rate,Population,Infection Rate,Susceptible Population,Rate of change of susceptible population,Action,Reward
583,27-Jul-20,Assam,32228,0,24040,79,32228,5.878372,0.170115,15.975457,2.717667,74.593521,0.245128,31205576,0.103276,31173348,-0.003663,99.896724,2.552567e+00
584,27-Jul-20,Delhi,130605,1,114875,3827,130606,23.822474,0.041977,8.594458,0.360771,87.955377,2.930187,16787941,0.777975,16657335,-0.006453,99.222025,1.527946e+06
585,27-Jul-20,Jharkhand,8275,0,3704,85,8275,1.509356,0.662534,16.888024,11.188892,44.761329,1.027190,32988134,0.025085,32979859,-0.001331,99.974915,4.706000e+06
586,27-Jul-20,Maharashtra,375796,3,213238,13656,375799,68.545564,0.014589,57.529182,0.839284,56.742567,3.633857,112374333,0.334417,111998534,-0.008420,99.665583,9.410055e+06
587,27-Jul-20,Nagaland,1339,0,549,4,1339,0.244233,4.094451,1.012879,4.147184,41.000747,0.298730,1978502,0.067677,1977163,-0.002529,99.932323,8.602806e+01


In [ ]:
samples = np.array(df['Action'])
from collections import Counter
counts = Counter(samples)
print(counts)
total = sum(counts.values())
print(total)
probability_mass = {k:v/total for k,v in counts.items()}
list(probability_mass.keys()) , list(probability_mass.values()), probability_mass.items() 

Counter({1: 280, 0: 72, 2: 55, 3: 55, 4: 27, 8: 17, 5: 13, 7: 13, 9: 12, 6: 8, 10: 7, 13: 6, 18: 5, 14: 4, 15: 3, 12: 2, 21: 2, 16: 2, 19: 1, 22: 1, 24: 1, 23: 1, 11: 1})
588


([0,
  1,
  2,
  3,
  5,
  4,
  7,
  8,
  6,
  10,
  9,
  12,
  13,
  14,
  15,
  18,
  19,
  22,
  24,
  23,
  21,
  16,
  11],
 [0.12244897959183673,
  0.47619047619047616,
  0.0935374149659864,
  0.0935374149659864,
  0.022108843537414966,
  0.04591836734693878,
  0.022108843537414966,
  0.02891156462585034,
  0.013605442176870748,
  0.011904761904761904,
  0.02040816326530612,
  0.003401360544217687,
  0.01020408163265306,
  0.006802721088435374,
  0.00510204081632653,
  0.008503401360544218,
  0.0017006802721088435,
  0.0017006802721088435,
  0.0017006802721088435,
  0.0017006802721088435,
  0.003401360544217687,
  0.003401360544217687,
  0.0017006802721088435],
 dict_items([(0, 0.12244897959183673), (1, 0.47619047619047616), (2, 0.0935374149659864), (3, 0.0935374149659864), (5, 0.022108843537414966), (4, 0.04591836734693878), (7, 0.022108843537414966), (8, 0.02891156462585034), (6, 0.013605442176870748), (10, 0.011904761904761904), (9, 0.02040816326530612), (12, 0.003401360544217

In [ ]:
for i in range(len(df)):
  df['Probability'] = df['Action']
df.head()

,Date,Name of State / UT,Total Confirmed cases (Indian National),Total Confirmed cases ( Foreign National ),Cured/Discharged/Migrated,Death,Total Confirmed cases,%share of infected cases,1 / (% share of infected cases),Population Share,(Population share) / (% share of infected cases),Recovery Rate,Death Rate,Population,Infection Rate,Susceptible Population,Rate of change of susceptible population,Action,Reward,Probability
0,10-Mar-20,Delhi,4,0,0,0,4,66.666667,0.015000,8.594458,0.128917,0.0,0.0,16787941,0.000024,16787937,0.000000,0,0.0,0
1,10-Mar-20,Maharashtra,2,0,0,0,2,33.333333,0.030000,57.529182,1.725875,0.0,0.0,112374333,0.000002,112374331,0.000000,0,0.0,0
2,11-Mar-20,Delhi,5,0,0,0,5,83.333333,0.012000,8.594458,0.103134,0.0,0.0,16787941,0.000030,16787936,-0.000006,1,0.0,1
3,11-Mar-20,Maharashtra,2,0,0,0,2,33.333333,0.030000,57.529182,1.725875,0.0,0.0,112374333,0.000002,112374331,0.000000,0,0.0,0
4,12-Mar-20,Delhi,6,0,0,0,6,35.294118,0.028333,8.594458,0.243510,0.0,0.0,16787941,0.000036,16787935,-0.000006,1,0.0,1


In [ ]:
df['Probability'].replace(list(probability_mass.keys()), list(probability_mass.values()), inplace=True )
df.head()

,Date,Name of State / UT,Total Confirmed cases (Indian National),Total Confirmed cases ( Foreign National ),Cured/Discharged/Migrated,Death,Total Confirmed cases,%share of infected cases,1 / (% share of infected cases),Population Share,(Population share) / (% share of infected cases),Recovery Rate,Death Rate,Population,Infection Rate,Susceptible Population,Rate of change of susceptible population,Action,Reward,Probability
0,10-Mar-20,Delhi,4,0,0,0,4,66.666667,0.015000,8.594458,0.128917,0.0,0.0,16787941,0.000024,16787937,0.000000,0,0.0,0.122449
1,10-Mar-20,Maharashtra,2,0,0,0,2,33.333333,0.030000,57.529182,1.725875,0.0,0.0,112374333,0.000002,112374331,0.000000,0,0.0,0.122449
2,11-Mar-20,Delhi,5,0,0,0,5,83.333333,0.012000,8.594458,0.103134,0.0,0.0,16787941,0.000030,16787936,-0.000006,1,0.0,0.476190
3,11-Mar-20,Maharashtra,2,0,0,0,2,33.333333,0.030000,57.529182,1.725875,0.0,0.0,112374333,0.000002,112374331,0.000000,0,0.0,0.122449
4,12-Mar-20,Delhi,6,0,0,0,6,35.294118,0.028333,8.594458,0.243510,0.0,0.0,16787941,0.000036,16787935,-0.000006,1,0.0,0.476190


In [ ]:
df_train = df[0:503] #(70%) (initially df1[0:3501])
df_test = df[503:588] #(30%)   (initialyy df1[3501:])
df_train.shape, df_test.shape
# df_train.tail(n=10)
# df_train['index'] = range(1, len(df_train) + 1)
# df_train = train_df.set_index("index")
# df_test['index'] = range(1, len(df_test) + 1)
# df_test = test_df.set_index("index")
# df_test.head(),df_train.head()

((503, 20), (85, 20))

In [ ]:

# generate sample data that could originate from previous random trial, e.g. AB test, for the CB to explore
## data here are equivalent to example in https://github.com/JohnLangford/vowpal_wabbit/wiki/Contextual-Bandit-Example
train_data = [{'action': 1, 'cost': 2, 'probability': 0.4, 'feature1': 'a', 'feature2': 'c', 'feature3': ''},
              {'action': 3, 'cost': 0, 'probability': 0.2, 'feature1': 'b', 'feature2': 'd', 'feature3': ''},
              {'action': 4, 'cost': 1, 'probability': 0.5, 'feature1': 'a', 'feature2': 'b', 'feature3': ''},
              {'action': 2, 'cost': 1, 'probability': 0.3, 'feature1': 'a', 'feature2': 'b', 'feature3': 'c'},
              {'action': 3, 'cost': 1, 'probability': 0.7, 'feature1': 'a', 'feature2': 'd', 'feature3': ''}]

train_df = pd.DataFrame(train_data)

## add index to df
train_df['index'] = range(1, len(train_df) + 1)
train_df = train_df.set_index("index")

# generate some test data that you want the CB to make decisions for, e.g. features describing new users, for the CB to exploit
test_data = [{'feature1': 'b', 'feature2': 'c', 'feature3': ''},
            {'feature1': 'a', 'feature2': '', 'feature3': 'b'},
            {'feature1': 'b', 'feature2': 'b', 'feature3': ''},
            {'feature1': 'a', 'feature2': '', 'feature3': 'b'}]

test_df = pd.DataFrame(test_data)

## add index to df
test_df['index'] = range(1, len(test_df) + 1)
test_df = test_df.set_index("index")

In [ ]:
# import vowpal wabbit's python wrapper
from vowpalwabbit import pyvw

# create python model - this stores the model parameters in the python vw object; here a contextual bandit with four possible actions
vw = pyvw.vw("--cb 25 --epsilon 0.25")

# use the learn method to train the vw model, train model row by row using a loop
for i in df_train.index:
  ## provide data to cb in requested format
  action = df_train['Action'].iloc[i]
  cost = 1/df_train['Reward'].iloc[i]
  probability = df_train['Probability'].iloc[i]
  feature1 = df_train['Death Rate'].iloc[i]
  feature2 = df_train['Recovery Rate'].iloc[i]
  feature3 = df_train['Susceptible Population'].iloc[i]
  feature4 = df_train['Population Share'].iloc[i]
  feature5 = df_train['Total Confirmed cases'].iloc[i]
  ## do the actual learning
  vw.learn(str(action)+":"+str(cost)+":"+str(probability)+" | "+str(feature1)+" "+str(feature2)+" "+str(feature3)+" "+str(feature4)+" "+str(feature5))
  # vw.learn(str(action)+":"+str(cost)+ "| "+str(feature1)+" "+str(feature2)+" "+str(feature3)+" "+str(feature4)+" "+str(feature5))

# use the same model object that was trained to perform predictions

# predict row by row and output results
choices = []
for j in range(len(df_test)):
  feature1 = df_test['Death Rate'].iloc[j]
  feature2 = df_test['Recovery Rate'].iloc[j]
  feature3 = df_test['Susceptible Population'].iloc[j]
  feature4 = df_test['Population Share'].iloc[j]
  feature5 = df_test['Total Confirmed cases'].iloc[j]
  # print(feature1)
  choice = vw.predict("| "+str(feature1)+" "+str(feature2)+" "+str(feature3)+" "+str(feature4)+" "+str(feature5))
  choices.append(choice)
  print(f"Action : {choice}\t  Date : {df_test['Date'].iloc[j]}\t Location : {df_test['Name of State / UT'].iloc[j]} \t")

# the CB assigns every instance to action 3 as it should per the cost structure of the train data; you can play with the cost structure to see that the CB updates its predictions accordingly

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


Action : 22	  Date : 11-Jul-20	 Location : Assam 	
Action : 11	  Date : 11-Jul-20	 Location : Delhi 	
Action : 23	  Date : 11-Jul-20	 Location : Jharkhand 	
Action : 24	  Date : 11-Jul-20	 Location : Maharashtra 	
Action : 23	  Date : 11-Jul-20	 Location : Nagaland 	
Action : 22	  Date : 12-Jul-20	 Location : Assam 	
Action : 11	  Date : 12-Jul-20	 Location : Delhi 	
Action : 24	  Date : 12-Jul-20	 Location : Jharkhand 	
Action : 24	  Date : 12-Jul-20	 Location : Maharashtra 	
Action : 23	  Date : 12-Jul-20	 Location : Nagaland 	
Action : 22	  Date : 13-Jul-20	 Location : Assam 	
Action : 11	  Date : 13-Jul-20	 Location : Delhi 	
Action : 24	  Date : 13-Jul-20	 Location : Jharkhand 	
Action : 24	  Date : 13-Jul-20	 Location : Maharashtra 	
Action : 23	  Date : 13-Jul-20	 Location : Nagaland 	
Action : 22	  Date : 14-Jul-20	 Location : Assam 	
Action : 11	  Date : 14-Jul-20	 Location : Delhi 	
Action : 24	  Date : 14-Jul-20	 Location : Jharkhand 	
Action : 24	  Date : 14-Jul-20	 Locatio

In [ ]:
new_set = set(choices) #Unique actions number suggested by model
print(new_set)
samples = choices
from collections import Counter
counts = Counter(samples)
print(counts)

{6, 11, 22, 23, 24}
Counter({24: 32, 22: 18, 23: 18, 11: 16, 6: 1})


In [ ]:
# BONUS: save and load the CB model
# save model
vw.save('cb.model')
del vw
# load from saved file
vw = pyvw.vw("--cb 4 -i cb.model")
print(vw.predict('| '))

In [ ]:
def sample_custom_pmf(pmf):
    total = sum(pmf)
    scale = 1/total
    pmf = [x * scale for x in pmf]
    draw = random.random()
    sum_prob = 0.0
    for index, prob in enumerate(pmf):
        sum_prob += prob
        if(sum_prob > draw):
            return index, prob

In [ ]:
def get_action(vw, context, actions):
    # vw_text_example = to_vw_example_format(context,actions)
for i in range()
    pmf = vw.predict(vw_text_example)
    chosen_action_index, prob = sample_custom_pmf(pmf)
    return actions[chosen_action_index], prob

In [ ]:
[A: 18 , D: 20 , J: 21 , M: 45 , N: 16,  A: 15 , D: 20 , J: 21 , M: 45 , N: 16, A: 18 , D: 20 , J: 21 , M: 45 , N: 16,.....] - 27 july

18 came 2 time 2/3 = 66.66 safe 1